# Camada Ouro - Enriquecimento e Dados Finais

Este notebook executa o enriquecimento dos dados da camada Prata, criando:
- Métricas avançadas
- Classificações e indicadores
- Tabelas agregadas para análise
- Dados prontos para consumo

## Estrutura da Camada Ouro
- `ouro/{dataset}/dt={data}/data.parquet`

In [ ]:
# Instalar dependências se necessário
!pip install -q pandas minio pyarrow matplotlib seaborn

In [ ]:
# Executar script de enriquecimento Ouro
exec(open('03_ouro_enriquecimento.py').read())

## Visualizar e analisar dados enriquecidos

In [ ]:
import pandas as pd
from minio import Minio
import io
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações
MINIO_SERVER_URL = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ROOT_USER = "admin"
MINIO_ROOT_PASSWORD = "1q2w3e4r"
BUCKET_NAME = "govbr"

minio_client = Minio(
    MINIO_SERVER_URL,
    access_key=MINIO_ROOT_USER,
    secret_key=MINIO_ROOT_PASSWORD,
    secure=True
)

# Listar arquivos Ouro
objects = minio_client.list_objects(BUCKET_NAME, prefix="ouro/", recursive=True)
ouro_files = list(objects)

print(f"Total de arquivos na camada Ouro: {len(ouro_files)}")
for obj in ouro_files:
    print(f"📁 {obj.object_name} ({obj.size/1024:.2f} KB)")

In [ ]:
# Visualizar estados enriquecidos
objects = list(minio_client.list_objects(BUCKET_NAME, prefix="ouro/dim_estados_enriquecida/", recursive=True))
if objects:
    latest = max(objects, key=lambda x: x.last_modified)
    response = minio_client.get_object(BUCKET_NAME, latest.object_name)
    df_estados = pd.read_parquet(io.BytesIO(response.read()))
    response.close()
    response.release_conn()
    
    print("Estados Enriquecidos:")
    print(f"Registros: {len(df_estados)}")
    print(df_estados.head(10))

In [ ]:
# Visualizar agregação por região
objects = list(minio_client.list_objects(BUCKET_NAME, prefix="ouro/agregacao_bpc_por_regiao/", recursive=True))
if objects:
    latest = max(objects, key=lambda x: x.last_modified)
    response = minio_client.get_object(BUCKET_NAME, latest.object_name)
    df_regiao = pd.read_parquet(io.BytesIO(response.read()))
    response.close()
    response.release_conn()
    
    print("Agregação BPC por Região:")
    print(df_regiao)
    
    # Gráfico
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df_regiao, x='regiao_nome', y='total_valor')
    plt.title('Total de Valor BPC por Região')
    plt.xlabel('Região')
    plt.ylabel('Valor Total (R$)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Top 10 municípios por valor BPC
objects = list(minio_client.list_objects(BUCKET_NAME, prefix="ouro/top_10_municipios_valor_bpc/", recursive=True))
if objects:
    latest = max(objects, key=lambda x: x.last_modified)
    response = minio_client.get_object(BUCKET_NAME, latest.object_name)
    df_top = pd.read_parquet(io.BytesIO(response.read()))
    response.close()
    response.release_conn()
    
    print("Top 10 Municípios por Valor BPC:")
    print(df_top)
    
    # Gráfico
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df_top, x='nome_municipio', y='valor')
    plt.title('Top 10 Municípios - Valor BPC')
    plt.xlabel('Município')
    plt.ylabel('Valor (R$)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()